# Implementasi Logika Fuzzy Mamdani dan Sugeno untuk Prediksi Indeks Kualitas Udara (AQI)

## 1. Import Library yang Diperlukan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import skfuzzy as fuzz
from skfuzzy import control as ctrl

## 2. Data Preprocessing

In [ ]:
# Membaca dataset
df = pd.read_csv('aqi_data_date.csv')

# Menampilkan informasi dataset
print("Informasi Dataset:")
print("-" * 50)
print(df.info())
print("\nStatistik Deskriptif:")
print("-" * 50)
print(df.describe())

# Mengecek missing values
print("\nMissing Values:")
print("-" * 50)
print(df.isnull().sum())

In [ ]:
# Visualisasi distribusi AQI Value
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='AQI Value', bins=30)
plt.title('Distribusi AQI Value')
plt.xlabel('AQI Value')
plt.ylabel('Frekuensi')
plt.show()

# Visualisasi Status AQI
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Status')
plt.title('Distribusi Status AQI')
plt.xticks(rotation=45)
plt.show()

## 3. Implementasi Fuzzy System Mamdani

In [ ]:
# Membuat variabel fuzzy untuk input dan output
aqi = np.arange(0, 501, 1)
status = np.arange(0, 1.1, 0.1)

# Membuat membership functions untuk input AQI
aqi_good = fuzz.trimf(aqi, [0, 50, 100])
aqi_moderate = fuzz.trimf(aqi, [50, 100, 150])
aqi_unhealthy = fuzz.trimf(aqi, [100, 150, 200])
aqi_very_unhealthy = fuzz.trimf(aqi, [150, 200, 300])
aqi_hazardous = fuzz.trimf(aqi, [200, 300, 500])

# Visualisasi membership functions
plt.figure(figsize=(10, 6))
plt.plot(aqi, aqi_good, 'b', linewidth=1.5, label='Good')
plt.plot(aqi, aqi_moderate, 'g', linewidth=1.5, label='Moderate')
plt.plot(aqi, aqi_unhealthy, 'y', linewidth=1.5, label='Unhealthy')
plt.plot(aqi, aqi_very_unhealthy, 'r', linewidth=1.5, label='Very Unhealthy')
plt.plot(aqi, aqi_hazardous, 'purple', linewidth=1.5, label='Hazardous')
plt.title('Membership Functions untuk AQI')
plt.xlabel('AQI Value')
plt.ylabel('Membership Degree')
plt.legend()
plt.grid(True)
plt.show()

## 3. Implementasi Fuzzy System Mamdani (Lanjutan)

In [ ]:
# Membuat membership functions untuk output Status
status_good = fuzz.trimf(status, [0, 0.2, 0.4])
status_moderate = fuzz.trimf(status, [0.3, 0.5, 0.7])
status_unhealthy = fuzz.trimf(status, [0.6, 0.8, 1.0])

# Visualisasi membership functions untuk output
plt.figure(figsize=(10, 6))
plt.plot(status, status_good, 'b', linewidth=1.5, label='Good')
plt.plot(status, status_moderate, 'g', linewidth=1.5, label='Moderate')
plt.plot(status, status_unhealthy, 'r', linewidth=1.5, label='Unhealthy')
plt.title('Membership Functions untuk Status Output')
plt.xlabel('Status Value')
plt.ylabel('Membership Degree')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Membuat fungsi untuk implementasi Fuzzy System Mamdani
def mamdani_fuzzy_system(aqi_value):
    # Fuzzifikasi input
    good_degree = fuzz.interp_membership(aqi, aqi_good, aqi_value)
    moderate_degree = fuzz.interp_membership(aqi, aqi_moderate, aqi_value)
    unhealthy_degree = fuzz.interp_membership(aqi, aqi_unhealthy, aqi_value)
    very_unhealthy_degree = fuzz.interp_membership(aqi, aqi_very_unhealthy, aqi_value)
    hazardous_degree = fuzz.interp_membership(aqi, aqi_hazardous, aqi_value)
    
    # Aplikasi rules fuzzy
    # Rule 1: IF AQI is Good THEN Status is Good
    rule1 = np.fmin(good_degree, status_good)
    
    # Rule 2: IF AQI is Moderate THEN Status is Moderate
    rule2 = np.fmin(moderate_degree, status_moderate)
    
    # Rule 3: IF AQI is Unhealthy THEN Status is Unhealthy
    rule3 = np.fmin(unhealthy_degree, status_unhealthy)
    
    # Rule 4: IF AQI is Very Unhealthy THEN Status is Unhealthy
    rule4 = np.fmin(very_unhealthy_degree, status_unhealthy)
    
    # Rule 5: IF AQI is Hazardous THEN Status is Unhealthy
    rule5 = np.fmin(hazardous_degree, status_unhealthy)
    
    # Agregasi output
    aggregated = np.fmax(rule1, np.fmax(rule2, np.fmax(rule3, np.fmax(rule4, rule5))))
    
    # Defuzzifikasi menggunakan metode centroid
    defuzzified = fuzz.defuzz(status, aggregated, 'centroid')
    
    # Konversi hasil defuzzifikasi ke status
    if defuzzified <= 0.4:
        return 'Good'
    elif defuzzified <= 0.7:
        return 'Moderate'
    else:
        return 'Unhealthy'

# Evaluasi model Mamdani
def evaluate_mamdani(df):
    predictions = []
    for aqi_value in df['AQI Value']:
        pred = mamdani_fuzzy_system(aqi_value)
        predictions.append(pred)
    
    # Menghitung akurasi
    accuracy = accuracy_score(df['Status'], predictions)
    
    # Menghitung F1-score
    f1 = f1_score(df['Status'], predictions, average='weighted')
    
    # Membuat confusion matrix
    cm = confusion_matrix(df['Status'], predictions)
    
    return accuracy, f1, cm, predictions

# Menjalankan evaluasi
accuracy_mamdani, f1_mamdani, cm_mamdani, predictions_mamdani = evaluate_mamdani(df)

print(f"Akurasi Model Mamdani: {accuracy_mamdani:.4f}")
print(f"F1-Score Model Mamdani: {f1_mamdani:.4f}")

# Visualisasi confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_mamdani, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Model Mamdani')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 4. Implementasi Fuzzy System Sugeno

In [ ]:
# Membuat fungsi untuk implementasi Fuzzy System Sugeno
def sugeno_fuzzy_system(aqi_value):
    # Fuzzifikasi input
    good_degree = fuzz.interp_membership(aqi, aqi_good, aqi_value)
    moderate_degree = fuzz.interp_membership(aqi, aqi_moderate, aqi_value)
    unhealthy_degree = fuzz.interp_membership(aqi, aqi_unhealthy, aqi_value)
    very_unhealthy_degree = fuzz.interp_membership(aqi, aqi_very_unhealthy, aqi_value)
    hazardous_degree = fuzz.interp_membership(aqi, aqi_hazardous, aqi_value)
    
    # Aplikasi rules fuzzy dengan output crisp
    # Rule 1: IF AQI is Good THEN Status = 0.2 (Good)
    rule1 = good_degree * 0.2
    
    # Rule 2: IF AQI is Moderate THEN Status = 0.5 (Moderate)
    rule2 = moderate_degree * 0.5
    
    # Rule 3: IF AQI is Unhealthy THEN Status = 0.8 (Unhealthy)
    rule3 = unhealthy_degree * 0.8
    
    # Rule 4: IF AQI is Very Unhealthy THEN Status = 0.9 (Unhealthy)
    rule4 = very_unhealthy_degree * 0.9
    
    # Rule 5: IF AQI is Hazardous THEN Status = 1.0 (Unhealthy)
    rule5 = hazardous_degree * 1.0
    
    # Menghitung weighted average
    numerator = rule1 + rule2 + rule3 + rule4 + rule5
    denominator = good_degree + moderate_degree + unhealthy_degree + very_unhealthy_degree + hazardous_degree
    
    if denominator == 0:
        return 'Moderate'  # Default value
    
    output = numerator / denominator
    
    # Konversi output ke status
    if output <= 0.4:
        return 'Good'
    elif output <= 0.7:
        return 'Moderate'
    else:
        return 'Unhealthy'

# Evaluasi model Sugeno
def evaluate_sugeno(df):
    predictions = []
    for aqi_value in df['AQI Value']:
        pred = sugeno_fuzzy_system(aqi_value)
        predictions.append(pred)
    
    # Menghitung akurasi
    accuracy = accuracy_score(df['Status'], predictions)
    
    # Menghitung F1-score
    f1 = f1_score(df['Status'], predictions, average='weighted')
    
    # Membuat confusion matrix
    cm = confusion_matrix(df['Status'], predictions)
    
    return accuracy, f1, cm, predictions

# Menjalankan evaluasi
accuracy_sugeno, f1_sugeno, cm_sugeno, predictions_sugeno = evaluate_sugeno(df)

print(f"Akurasi Model Sugeno: {accuracy_sugeno:.4f}")
print(f"F1-Score Model Sugeno: {f1_sugeno:.4f}")

# Visualisasi confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_sugeno, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Model Sugeno')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 5. Perbandingan Model Mamdani dan Sugeno

In [ ]:
# Membuat perbandingan metrik evaluasi
metrics = pd.DataFrame({
    'Model': ['Mamdani', 'Sugeno'],
    'Akurasi': [accuracy_mamdani, accuracy_sugeno],
    'F1-Score': [f1_mamdani, f1_sugeno]
})

print("Perbandingan Metrik Evaluasi:")
print(metrics)

# Visualisasi perbandingan
plt.figure(figsize=(10, 6))
x = np.arange(2)
width = 0.35

plt.bar(x - width/2, metrics['Akurasi'], width, label='Akurasi')
plt.bar(x + width/2, metrics['F1-Score'], width, label='F1-Score')

plt.xlabel('Model')
plt.ylabel('Score')
plt.title('Perbandingan Kinerja Model Mamdani dan Sugeno')
plt.xticks(x, metrics['Model'])
plt.legend()
plt.show()